In [3]:
#!pip install git+https://github.com/tensorflow/examples.git

<span style = 'color : red'>Ici on installe tensorflow depuis git hub : Bibliothèque de machine learning axée sur la reconnaissance d'images (Computer vision). On y ajoute d'autres packages qui seront utiles pour le programme
    
    

# Reco

In [2]:
# GENERAL

import os # Module qui permet à Python d'intéragir avec le système d'exploitation, par exemple pour obtenir des chemins
# de fichiers.
import os.path
from pathlib import Path #Ajout d'autres fonctionnalités liés à la gestion de fichiers.
import time #Module fournit des fonctions liées au temps comme la fonction time() 
import pandas as pd #Module très important dans l'analyse de données (sous forme de dataframes)
import numpy as np #Module classique mathématique
import random
import time
import math
import glob #Module permettant de naviguer/sélectionner un certains type de fichiers selon des critères définis.
import cv2 #Module important pour le traitement d'images
from IPython.display import clear_output #API publique pour les outils d’affichage dans IPython.


import matplotlib.pyplot as plt #Module classique pour le tracé de graphiques.
import matplotlib.animation as animation
from IPython.display import HTML #Permet à Python de supporter les objets en HTML


# I PACKAGES (on )

import tensorflow as tf
import tensorflow.keras #Keras est une API de réseau de neurones écrite en langage Python. Il s’agit d’une bibliothèque 
#Open Source, exécutée par-dessus des frameworks tels que TensorFlow comme ici.


from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN,\
LSTM, GlobalAveragePooling2D, SeparableConv2D, ZeroPadding2D, Convolution2D, ZeroPadding2D,Reshape, Conv2DTranspose, LeakyReLU, ReLU, Conv2DTranspose
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow_examples.models.pix2pix import pix2pix

# WARNINGS

from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning) 
filterwarnings("ignore", category=UserWarning)


AUTOTUNE = tf.data.AUTOTUNE

ModuleNotFoundError: No module named 'tensorflow'

# BEFORE TRAINING : Initialisation du modèle


In [ ]:
BYZANTINE_ARC_PATH = Path("../input/architecture-dataset/arcDataset/Byzantine architecture")
GOTHIC_ARC_PATH = Path("../input/architecture-dataset/arcDataset/Gothic architecture")

<span style='color : red'>
    Ici on associe à la variable BYZANTINE_ARC_PATH le chemin d'accès pour pouvoir récuperer la database. ( idm pour la data base gothique )

In [ ]:
BYZANTINE_JPG_LIST = list(BYZANTINE_ARC_PATH.glob(r"*.jpg")) #La méthode .glob() permet de sélectionner tout les élements commençant par "*.jpg"
GOTHIC_JPG_LIST = list(GOTHIC_ARC_PATH.glob(r"*.jpg"))

<span style='color : red'>Mise sous forme de liste les différentes images

In [ ]:
print(len(BYZANTINE_JPG_LIST))
print(len(GOTHIC_JPG_LIST))

<span style='color : red'>Affiche la longeur des listes pour savoir le nombre de ressources disponibles

In [ ]:
GOTHIC_JPG_LIST = GOTHIC_JPG_LIST[:83]

<span style='color : red'>On tronque la database Gothique aux 84 premiers élements car la liste gothique est plus longue que la liste byzantine

In [1]:
print(len(GOTHIC_JPG_LIST) == len(BYZANTINE_JPG_LIST))

NameError: name 'GOTHIC_JPG_LIST' is not defined

<span style='color : red'>verifie si les liste sont de mm longueur

In [ ]:
GOTHIC_SERIES = pd.Series(GOTHIC_JPG_LIST,name="GOTHIC").astype(str) 
BYZANTINE_SERIES = pd.Series(BYZANTINE_JPG_LIST,name="BYZANTINE").astype(str)

MAIN_ARC = pd.concat([BYZANTINE_SERIES,GOTHIC_SERIES],axis=1)

<span style='color : red'>convertie les type des series du dataframe en string. Et on les concatène selon axis 1 (selon les abscisses)

In [ ]:
MAIN_ARC

<span style='color : red'>affiche le dataframe

In [ ]:
figure,axis = plt.subplots(3,3,figsize=(14,14)) #Création du plot à 3 à 4

for i_ind,i_ops in enumerate(axis.flat):
    
    IMAGE_READING = cv2.cvtColor(cv2.imread(MAIN_ARC["BYZANTINE"][i_ind]),cv2.COLOR_BGR2RGB)
    i_ops.set_title("BYZANTINE")
    i_ops.axis("off")
    i_ops.imshow(IMAGE_READING)
    
plt.tight_layout()
plt.show()    

In [ ]:
figure,axis = plt.subplots(3,3,figsize=(14,14))

for i_ind,i_ops in enumerate(axis.flat):
    
    IMAGE_READING = cv2.cvtColor(cv2.imread(MAIN_ARC["GOTHIC"][i_ind]),cv2.COLOR_BGR2RGB)
    i_ops.set_title("GOTHIC")
    i_ops.axis("off")
    i_ops.imshow(IMAGE_READING)
    
plt.tight_layout()
plt.show()    

<span style='color : red'>affiche les images

# PREPARING : Préparation de l'algorithme de reconnaissance 


In [ ]:
BUFFER_SIZE = 400
BATCH_SIZE = 1
IMG_WIDTH = 256
IMG_HEIGHT = 256
OUTPUT_CHANNELS = 3
LAMBDA = 100
EPOCHS = 300
TYPE_ARRAY = "float32"

In [ ]:
BYZANTINE_TARGET = []
GOTHIC_TARGET = []

for i_byzantine,i_gothic in zip(MAIN_ARC.BYZANTINE,MAIN_ARC.GOTHIC):
    
    BYZ_READING = cv2.cvtColor(cv2.imread(i_byzantine),cv2.COLOR_BGR2RGB) #On convertit nos images d'églises byzantines
    # de BGR à RGB permettant aux images d'être compatibles avec les librairies comme TensorFlow
    
    et
    GOT_READING = cv2.cvtColor(cv2.imread(i_gothic),cv2.COLOR_BGR2RGB)
    
    BYZ_RESIZE = cv2.resize(BYZ_READING,(IMG_WIDTH,IMG_HEIGHT))
    GOT_RESIZE = cv2.resize(GOT_READING,(IMG_WIDTH,IMG_HEIGHT))
    
    BYZ_NORMALIZE = (BYZ_RESIZE / 127.5) - 1
    GOT_NORMALIZE = (GOT_RESIZE / 127.5) - 1
    
    BYZANTINE_TARGET.append(BYZ_NORMALIZE)
    GOTHIC_TARGET.append(GOT_NORMALIZE)

In [ ]:
print(np.shape(np.array(BYZANTINE_TARGET)))
print(np.shape(np.array(GOTHIC_TARGET)))

In [ ]:
figure,axis = plt.subplots(3,3,figsize=(15,15))

for x_number,x_operators in enumerate(axis.flat):
    
    EXAMPLE_PICKING = BYZANTINE_TARGET[x_number]
    
    x_operators.set_title("BYZANTINE")
    x_operators.axis("off")
    x_operators.imshow(EXAMPLE_PICKING)
    
plt.tight_layout()
plt.show()

In [ ]:
figure,axis = plt.subplots(3,3,figsize=(15,15))

for x_number,x_operators in enumerate(axis.flat):
    
    EXAMPLE_PICKING = GOTHIC_TARGET[x_number]
    
    x_operators.set_title("GOTHIC")
    x_operators.axis("off")
    x_operators.imshow(EXAMPLE_PICKING)
    
plt.tight_layout()
plt.show()

In [ ]:
ARRAY_BYZ = np.array(BYZANTINE_TARGET,dtype=TYPE_ARRAY)
ARRAY_GOT = np.array(GOTHIC_TARGET,dtype=TYPE_ARRAY)

In [ ]:
print(ARRAY_BYZ.shape)
print(ARRAY_GOT.shape)

In [ ]:
BYZANTINE_TENSOR = tf.data.Dataset.from_tensor_slices(ARRAY_BYZ).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
GOTHIC_TENSOR = tf.data.Dataset.from_tensor_slices(ARRAY_GOT).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
print(BYZANTINE_TENSOR.element_spec)
print(GOTHIC_TENSOR.element_spec)

# LAYER FUNCTIONS : Fonctions couches de notre réseaux de neurones


In [ ]:
def DOWN_SAMPLE_CREATION(filters,size,apply_batchnorm=True):
    
    Initializer_Kernel = tf.random_normal_initializer(0.,0.02)
    
    Model = Sequential() #On crée un modèle séquentiel dans la variable Model grâce a Keras
    #Ce modèle de réseaux de neurones est séquentiel (chaque couche possède un input et un output)
    
    Model.add(Conv2D(filters,size,
                     strides=2,
                     padding="same",
                     kernel_initializer=Initializer_Kernel,
                     use_bias=False))
    
    if apply_batchnorm:
        Model.add(BatchNormalization())
        
        
    return Model

In [ ]:
DOWN_MODEL = DOWN_SAMPLE_CREATION(3,4)
RESULT_DOWN = DOWN_MODEL(tf.expand_dims(ARRAY_BYZ[0],0))

In [ ]:
print(RESULT_DOWN.shape)

In [ ]:
def UP_SAMPLE_CREATION(filters,size,apply_dropout=False):
    
    Initializer_Kernel = tf.random_normal_initializer(0.,0.02)
    
    Model = Sequential()
    
    Model.add(Conv2DTranspose(filters,size,
                     strides=2,
                     padding="same",
                     kernel_initializer=Initializer_Kernel,
                     use_bias=False))
    
    Model.add(BatchNormalization())
    
    
    if apply_dropout:
        
        Model.add(Dropout(0.3))
        
    Model.add(ReLU())
    
    return Model
    

In [ ]:
UP_MODEL = UP_SAMPLE_CREATION(3,4)
RESULT_UP = UP_MODEL(RESULT_DOWN)

In [ ]:
print(RESULT_UP.shape)

# GENERATOR et DISCRIMATOR : 

In [ ]:
def MODEL_GENERATOR(w_shape,h_shape,dim_shape):
    
    INPUT = tf.keras.layers.Input(shape=[w_shape,h_shape,dim_shape]) #On initial la première couche (layers) dans le INPUT
    #avec les dimensions w_shape, h_shape, dim_shape.
    
    DOWN_STACK = [
        
        DOWN_SAMPLE_CREATION(64,4,apply_batchnorm=False),
        DOWN_SAMPLE_CREATION(128,4),
        DOWN_SAMPLE_CREATION(256,4),
        DOWN_SAMPLE_CREATION(512,4),
        DOWN_SAMPLE_CREATION(512,4),
        DOWN_SAMPLE_CREATION(512,4),
        DOWN_SAMPLE_CREATION(512,4),
        DOWN_SAMPLE_CREATION(512,4),
    ]
    
    
    UP_STACK = [
        
        UP_SAMPLE_CREATION(512,4,apply_dropout=True),
        UP_SAMPLE_CREATION(512,4,apply_dropout=True),
        UP_SAMPLE_CREATION(512,4,apply_dropout=True),
        UP_SAMPLE_CREATION(512,4),
        UP_SAMPLE_CREATION(256,4),
        UP_SAMPLE_CREATION(128,4),
        UP_SAMPLE_CREATION(64,4),
    ]
    
    
    INITIALIZER_KERNEL = tf.random_normal_initializer(0.,0.02)
    
    LAST_LAY = Conv2DTranspose(OUTPUT_CHANNELS,
                               4,
                               strides=2,padding="same",
                              kernel_initializer=INITIALIZER_KERNEL,
                              activation="tanh")
    
    x = INPUT
    
    skips_lay = []
    
    for x_down in DOWN_STACK:
        
        x = x_down(x)
        skips_lay.append(x)
        
    skips_lay = reversed(skips_lay[:-1])
    
    for x_up,skipping in zip(UP_STACK,skips_lay):
        
        x = x_up(x)
        x = tf.keras.layers.Concatenate()([x,skipping])
        
    x = LAST_LAY(x)
    
    
    return tf.keras.Model(inputs=INPUT,outputs=x)

In [ ]:
Generator_Model = MODEL_GENERATOR(IMG_WIDTH,IMG_HEIGHT,OUTPUT_CHANNELS)

In [ ]:
tf.keras.utils.plot_model(Generator_Model, show_shapes=True, dpi=64)

In [ ]:
OUTPUT_GENERATOR_EXAMPLE = Generator_Model(ARRAY_BYZ[0][tf.newaxis,...],training=False)

In [ ]:
plt.figure(figsize=(8,8))
plt.axis("off")
plt.imshow(OUTPUT_GENERATOR_EXAMPLE[0])
plt.show()

In [ ]:
def MODEL_DISCRIMINATOR(w_shape,h_shape,dim_shape):
    
    INITIALIZER_KERNEL = tf.random_normal_initializer(0.,0.02)
    
    INPUT = tf.keras.layers.Input(shape=[w_shape,h_shape,dim_shape],name="INPUT_IMAGE")
    TARGET = tf.keras.layers.Input(shape=[w_shape,h_shape,dim_shape],name="TARGET_IMAGE")
    
    
    x = tf.keras.layers.concatenate([INPUT,TARGET])
    
    DOWN_L_1 = DOWN_SAMPLE_CREATION(64,4,False)(x)
    DOWN_L_2 = DOWN_SAMPLE_CREATION(64,4)(DOWN_L_1)
    DOWN_L_3 = DOWN_SAMPLE_CREATION(64,4)(DOWN_L_2)
    
    ZERO_PADDING_1 = tf.keras.layers.ZeroPadding2D()(DOWN_L_3)
    CONV2D_1 = Conv2D(512,
                      4,
                      strides=1,
                      kernel_initializer=INITIALIZER_KERNEL,
                     use_bias=False)(ZERO_PADDING_1)
    
    BATCHNORM_1 = BatchNormalization()(CONV2D_1)
    LEAKY_RELU_1 = LeakyReLU()(BATCHNORM_1)
    
    ZERO_PADDING_2 = tf.keras.layers.ZeroPadding2D()(LEAKY_RELU_1)
    
    LAST_LAY = Conv2D(1,
                      4,
                      strides=1,
                      kernel_initializer=INITIALIZER_KERNEL)(ZERO_PADDING_2)
    
    
    return tf.keras.Model(inputs=[INPUT,TARGET],outputs=LAST_LAY)

In [ ]:
Discriminator_Model = MODEL_DISCRIMINATOR(IMG_WIDTH,IMG_HEIGHT,OUTPUT_CHANNELS)

In [ ]:
tf.keras.utils.plot_model(Discriminator_Model, show_shapes=True, dpi=64)

In [ ]:
OUTPUT_DISCRIMINATOR_EXAMPLE = Discriminator_Model([ARRAY_BYZ[0][tf.newaxis,...],OUTPUT_GENERATOR_EXAMPLE],
                                                   training=False)

In [ ]:
plt.figure(figsize=(8,8))
plt.axis("off")
plt.imshow(OUTPUT_DISCRIMINATOR_EXAMPLE[0,...,-1],cmap='RdBu_r')
plt.show()

# LOSS FUNCTION : Fonction de coût de l'algorithme


In [ ]:
LOSS_FUNCTION = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [ ]:
def Generator_Loss(discriminator_output,generator_output,target):
    
    GAN_LOSS = LOSS_FUNCTION(tf.ones_like(discriminator_output),discriminator_output)
    
    L1_MEAN_ABS_ERROR = tf.reduce_mean(tf.abs(target-generator_output))
    
    TOTAL_LOSS = GAN_LOSS + (LAMBDA * L1_MEAN_ABS_ERROR)
    
    return TOTAL_LOSS,GAN_LOSS,L1_MEAN_ABS_ERROR

In [ ]:
def Discriminator_Loss(real_out,generated_out):
    
    REAL_LOSS = LOSS_FUNCTION(tf.ones_like(real_out),real_out)
    GEN_LOSS = LOSS_FUNCTION(tf.zeros_like(generated_out),generated_out)
    
    TOTAL_LOSS = REAL_LOSS + GEN_LOSS
    
    return TOTAL_LOSS

# OPTIMIZER AND GENERATED IMAGE

In [ ]:
GENERATOR_OPTIMIZER = tf.keras.optimizers.Adam(lr=0.0003, beta_1=0.5)
DISCRIMINATOR_OPTIMIZER = tf.keras.optimizers.Adam(lr=0.0003, beta_1=0.5)

In [ ]:
os.mkdir("./TRIP")
def generate_images_for_example(model, test_input, tar, number_i):
    
    prediction = model(test_input, training=True)
    
    RESULT_IN_OUT_LIST = [test_input[0], tar[0], prediction[0]]
    
    
    figure,axis = plt.subplots(1, 3,figsize=(14,14))

    axis[0].imshow(RESULT_IN_OUT_LIST[0] * 0.5 + 0.5)
    axis[0].axis('off')

    axis[1].imshow(RESULT_IN_OUT_LIST[1] * 0.5 + 0.5)
    axis[1].axis('off')

    axis[2].imshow(RESULT_IN_OUT_LIST[2] * 0.5 + 0.5)
    axis[2].axis('off')
    plt.savefig('./TRIP/trip_{:04d}.png'.format(number_i))
    plt.tight_layout()
    plt.show()

In [ ]:
os.mkdir("./SINGLE")
def generate_images_single(model, test_input, tar, number_i):
    
    prediction = model(test_input, training=True)
    
    plt.figure(figsize=(15, 15))
    plt.imshow(prediction[0] * 0.5 + 0.5)
    plt.savefig('./SINGLE/single_{:04d}.png'.format(number_i))
    plt.axis('off')
    plt.tight_layout()    
    plt.show()

In [ ]:
def generate_images(model, test_input, tar, number_i):
    
    prediction = model(test_input, training=True)
    
    plt.figure(figsize=(15, 15))
    plt.imshow(prediction[0] * 0.5 + 0.5)
    plt.savefig('output_{:04d}.png'.format(number_i))
    plt.axis('off')
    plt.tight_layout()    
    plt.show()

In [ ]:
for x_input,x_output in zip(BYZANTINE_TENSOR.take(1),GOTHIC_TENSOR.take(1)):
    
    generate_images_for_example(Generator_Model,x_input,x_output,1000000)

In [ ]:
for x_input,x_output in zip(BYZANTINE_TENSOR.take(1),GOTHIC_TENSOR.take(1)):
    
    generate_images(Generator_Model,x_input,x_output,1000000)

# TRAINING FUNCTION

In [ ]:
@tf.function

def Training_Step(INPUT_IMG,TARGET_IMG,EPOCH):
    
    with tf.GradientTape() as GEN_TAPE, tf.GradientTape() as DISC_TAPE:
        
        GENERATOR_OUT = Generator_Model(INPUT_IMG,training=True)
        
        DISCRIMINATOR_REAL = Discriminator_Model([INPUT_IMG,TARGET_IMG],training=True)
        DISCRIMINATOR_GENERATED = Discriminator_Model([INPUT_IMG,GENERATOR_OUT],training=True)
        
        GEN_TOTAL_LOSS,GEN_GENERATOR_LOSS, GEN_L1_MEAN_ABS = Generator_Loss(DISCRIMINATOR_GENERATED,
                                                                           GENERATOR_OUT,
                                                                            TARGET_IMG)
        
        DISC_LOSS = Discriminator_Loss(DISCRIMINATOR_REAL,DISCRIMINATOR_GENERATED)
        
        
    Generator_Gradients = GEN_TAPE.gradient(GEN_TOTAL_LOSS,Generator_Model.trainable_variables)
    Discriminator_Gradients = DISC_TAPE.gradient(DISC_LOSS,Discriminator_Model.trainable_variables)
    
    GENERATOR_OPTIMIZER.apply_gradients(zip(Generator_Gradients,Generator_Model.trainable_variables))
    DISCRIMINATOR_OPTIMIZER.apply_gradients(zip(Discriminator_Gradients,Discriminator_Model.trainable_variables))

#### YOU CAN USE AS A FUNCTION IF YOU WANT

In [ ]:
def Training_Start(TRAIN_DATA,TARGET_DATA,EPOCH):
    
    EXAMPLE_INP,EXAMPLE_TAR = next(iter(TRAIN_DATA.take(1))),next(iter(TARGET_DATA.take(1)))
    
    Starting_Time = time.time()
    
    for epc, (input_image_x,target_x) in enumerate(zip(TRAIN_DATA,TARGET_DATA)):
        
        if (epc) % 1000 == 0:
            
            clear_output(wait=True)
            
            if epc != 0:
                print(f'TIME FOR 1000 STEPS: {time.time()-Starting_Time:.2f} sec\n')
                
            Starting_Time = time.time()
            
            generate_images(Generator_Model,EXAMPLE_INP,EXAMPLE_TAR)
            
        Training_Step(input_image_x,target_x,epc)
            
        if (epc + 1) % 10 == 0:
            print("-",end="",flush=True)

# TRAINING

In [ ]:
EXAMPLE_INP,EXAMPLE_TAR = next(iter(BYZANTINE_TENSOR.take(1))),next(iter(GOTHIC_TENSOR.take(1)))

counting_img = 0

for epoch in range(EPOCHS):

    n = 0
    
    for image_x, image_y in tf.data.Dataset.zip((BYZANTINE_TENSOR, GOTHIC_TENSOR)):
        
        Training_Step(image_x, image_y, epoch)
        
        if n % 10 == 0:
            
            print ('.', end='')
        n += 1

    clear_output(wait=True)

    generate_images(Generator_Model,
                    image_x,
                    image_y,
                    counting_img)
    
    generate_images_single(Generator_Model,
                           EXAMPLE_INP,
                           EXAMPLE_TAR,
                           counting_img)
    
    generate_images_for_example(Generator_Model,
                                image_x,
                                image_y,
                                counting_img)
    
    counting_img = counting_img + 1

In [ ]:
#Training_Start(BYZANTINE_TENSOR,GOTHIC_TENSOR,EPOCHS) # İF YOU WANT TO USE AS A FUNCTION, RUN THIS LINE INSTEAD OF RUNNING PREVIOUS

In [ ]:
SINGLE_TARGET_PATH = Path("./TRIP")
SINGLE_JPG_TARGET = list(SINGLE_TARGET_PATH.glob(r"*.png"))
SERIES_OUTPUT_TARGET = pd.Series(SINGLE_JPG_TARGET,name="SERIES_OUTPUT").astype(str)

In [ ]:
print(len(SERIES_OUTPUT_TARGET))

In [ ]:
TARGET_PATH_READING = []

for x_im in SERIES_OUTPUT_TARGET.values:

    IMG_TARGET = cv2.cvtColor(cv2.imread(x_im),cv2.COLOR_BGR2RGB)
    TARGET_PATH_READING.append(IMG_TARGET)

In [ ]:
TARGET_PATH_READING = TARGET_PATH_READING[:100]

In [ ]:
def displaying_resource(source):
    
    figure = plt.figure(figsize=(14,14))
    
    Image_List = []
    plt.style.use("dark_background")
    for counting,indexing in enumerate(source):
        
        
        Read_IMG = plt.imshow(indexing, animated=True)
        plt.axis('off')
        Image_List.append([Read_IMG])

    Animation_Func = animation.ArtistAnimation(figure, Image_List, interval=244, repeat_delay=1000)
    
    plt.close()
    
    return Animation_Func

In [ ]:
HTML(displaying_resource(TARGET_PATH_READING).to_html5_video())